In [12]:
# Import kebanyakan dari hugging face

from datasets import load_dataset, DatasetDict, Dataset

# from transformers import (
#     AutoTokenizer,
#     AutoConfig, 
#     AutoModelForSequenceClassification,
#     DataCollatorWithPadding,
#     TrainingArguments,
#     Trainer)

# from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
# import evaluate
# import torch
import numpy as np
import random
import pandas as pd

# for GPT-2 model
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling


In [ ]:
# Model
model_name = "gpt2"  # 'gpt2-medium or gpt2-large'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tambahkan special tokens untuk format haiku 
# Seperti tanda baca untuk model memahami struktur teks generate (haiku)
special_tokens = {
    'pad_token': '<|pad|>', # mengisi bagian yang kosong dengan pad, harapannya panjangnya jadi sama
    'bos_token': '<|startoftext|>', # menandai awal dari haiku (konteks awal)
    'eos_token': '<|endoftext|>', # konteks akhir, menandai haiku sudah selesai
}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Coba implementasikan menggunakan LoRA kalau perlu
lora_config = LoraConfig(
    r=8,  # rank dari matriks adaptasi
    lora_alpha=32,  # scaling factor
    target_modules=["c_attn", "c_proj"],  # layer yang akan dimodifikasi
    lora_dropout=0.1,
    task_type=TaskType.CAUSAL_LM
)

# Aplikasikan LoRA ke model GPT-2
model = get_peft_model(model, lora_config)

In [15]:
# Load Dataset
df = pd.read_parquet("dataset2.parquet")
df.head()

,source,text,text_phonemes,keywords,keyword_phonemes,gruen_score,text_punc
0,bfbarry,Delicate savage. / You'll never hold the cinde...,deh|lax|kaxt sae|vaxjh / yuwl neh|ver hhowld d...,cinder,sihn|der,0.639071,None
1,bfbarry,A splash and a cry. / Words pulled from the ri...,ax splaesh aend ax kray / werdz puhld frahm dh...,the riverside,dhax rih|ver|sayd,0.563353,None
2,bfbarry,"Steamy, mist rising. / Rocks receiving downwar...",stiy|miy mihst ray|zaxng / raaks rax|siy|vaxng...,mist rising,mihst ray|zaxng,0.538326,None
3,bfbarry,You were broken glass. / But I touched you eve...,yuw wer brow|kaxn glaes / baht ay tahcht yuw i...,broken glass,brow|kaxn glaes,0.703446,None
4,bfbarry,Eyes dance with firelight. / The Moon and I ar...,ayz daens wihdh faxr|layt / dhax muwn aend ay ...,eyes dance,ayz daens,0.830985,None
